In [1]:
!pip install requests --upgrade --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install beautifulsoup4 --upgrade --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [446]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# sometimes websites stop you from extracting the data for some reason. It can be due to some authentication errors.

needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
response = requests.get("https://www.themoviedb.org/movie", headers = needed_headers )
response.status_code

200

In [7]:
dwn_content = response.text
len(dwn_content)
dwn_content[:500]

'<!DOCTYPE html>\n<html lang="it" class="no-js">\n  <head>\n    <title>Film Popolari &#8212; The Movie Database (TMDB)</title>\n    <meta http-equiv="cleartype" content="on">\n    <meta charset="utf-8">\n    <meta name="keywords" content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast">\n    <meta name="mobile-web-app-capable" content="yes">\n    <meta name="apple-mobile-web-app-capable" content="yes">\n    <meta name="viewpor'

In [13]:
test_doc = BeautifulSoup(dwn_content, 'html.parser')
type(test_doc)
test_doc.find('img')

<img alt="The Movie Database (TMDB)" height="20" src="/assets/2/v4/logos/v2/blue_short-8e7b30f73a4020692ccca9c88bafe5dcb6f8a62a4c6bc55cd9ba82bb2cd95f6c.svg" width="154"/>

In [16]:

def get_page_content(url):
    # In this case , we are going to give request.get function headers to avoid the Status code Error 403

    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(url, headers = get_headers )
    # we are going to raise exception here if status code gives any value other than 200.
    if not response_page.ok:
        raise Exception ("Failed to request the data. Status Code:- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc_page = BeautifulSoup(page_content, "html.parser")
        return doc_page
popular_shows_url = "https://www.themoviedb.org/movie"
doc = get_page_content(popular_shows_url)
doc.title.text

'Film Popolari — The Movie Database (TMDB)'

In [24]:
# Now that we know the class let's trty to get the title of the first movie. 

doc.find_all('div', {'class': 'card style_1'})[0].h2.text

'Rebel Moon - Parte 2: La sfregiatrice'

In [29]:
doc.find_all('div', {'class': 'user_score_chart'})[0]['data-percent']

'61'

In [500]:
def empty_dict():
    scraped_dict = {  
                    'Title': [],
                    'Genre': [],
                    'Duration': None,
                    'Release_date': None,
                    'User_rating': 0.0,
                    'Tagline': None,
                    'Description': None,
                    'Directors': None,
                    'Novel': None,
                    'Writers': None,
                    'Screenplay': None,
                    'State': None,
                    'Language': None,
                    'Budget': 0.0,
                    'Revenue': 0.0,
                    'Keywords': [],
                    }
    return scraped_dict

In [463]:
def user_score_info(tag_user_score, i, scraped_dict):
    if tag_user_score[i]['data-percent'] == '0':
        scraped_dict['User_rating'].append('Not rated yet')
    else:
        scraped_dict['User_rating'].append(tag_user_score[i]['data-percent'])

In [36]:
doc.find_all('div', {'class': 'card style_1'})[0].h2.a['href']

'/movie/934632-rebel-moon-part-two-the-scargiver'

In [38]:
def get_show_info(doc_page):
    base_link_1 = "https://www.themoviedb.org"
    tag_title = tag_premired_date = tag_shows_page = doc_page.find_all('div', {'class': 'card style_1'})
    tag_user_score = doc_page.find_all('div', {"user_score_chart"}) 
    
    doc_2_list = []
    for link in tag_shows_page:
        # here we are creating the list of all the individual pages of the shows which will come handy in other functions. 
        doc_2_list.append(get_page_content("https://www.themoviedb.org" + link.h2.a['href']))
       # we are going to have the function to return the list of all the information as elements. 
    return tag_title, tag_user_score, doc_2_list

In [42]:
#lets download and get the html of the individual page of the show 'what if...?' with the function get_page_content(). 
doc_2 = get_page_content("https://www.themoviedb.org/movie/693134")

In [578]:
def get_genres(doc2_page):
    genres_tags = doc2_page.find('span', {"class": "genres"}).find_all('a')
    check_genre =[]
    
    for tag in genres_tags:
        check_genre.append(tag.text)
    return check_genre

In [579]:
import re

def get_duration(doc2_page):
    # Estrae il testo del tag che contiene la durata
    duration_text = doc2_page.find('span', {"class": "runtime"}).text
    
    # Inizializza la variabile dei minuti
    minutes = 0
    
    # Usa espressioni regolari per trovare tutti i match di numeri seguiti da 'h' o 'm'
    parts = re.findall(r'(\d+)([hm])', duration_text)
    
    # Itera sui risultati trovati per calcolare i minuti totali
    for number, unit in parts:
        if unit == 'h':
            minutes += int(number) * 60  # Converti ore in minuti
        elif unit == 'm':
            minutes += int(number)  # Aggiungi i minuti direttamente
    return minutes


In [478]:
tag_tagline = doc_2.find('h3',{"class": 'tagline'})
def tagline_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('h3',{"class": 'tagline'}):
        scraped_dict['Tagline'] = doc_2_list[i].find('h3',{"class": 'tagline'}).text
    else:
        scraped_dict['Tagline'] = "No Tagline"

In [600]:
# i here denotes the the element of the list type variable``doc2_page`` that contains different doc pages.

def get_show_cast(doc2_page):
    cast_tags = doc2_page.find_all('li', {'class': 'card'})
    cast_lis = []
    
    for t in cast_tags:
         cast_lis.append(t.p.text)
    
    return cast_lis

In [143]:
def get_release(doc_2_list, i):
    # Ottieni il testo dall'elemento HTML
    release_text = doc_2_list[i].find('span', {"class": "release"}).text
    # Split basato su spazio seguito da una parentesi aperta '('
    release_date = release_text.split(' (')[0]
    return release_date


In [481]:
def get_description(doc2_page, i):
    
    description_tag = doc_2_list_[0].find('div', class_='overview')
    
    # Estrai il testo, se il tag è stato trovato
    if description_tag:
        description_text = description_tag.text.strip()
    else:
        description_text = "Descrizione non trovata."
    
    # Stampa o restituisci il testo della descrizione
    return description_text


In [587]:
def get_directors(doc2_page):
    
    description_tag = doc2_page.findAll('li', class_='profile')
    # Estrai il testo, se il tag è stato trovato
    directors = "Director not found"
    if description_tag:
        numgiri=0
        for profile in description_tag:
            # Cerca un tag 'p' con classe 'character' che contiene la parola 'Director'
            if 'Director' in profile.find('p', class_='character').text:
                if numgiri == 0:
                    directors = profile.find('p').text.strip()
                else:
                    directors = directors + ',' + profile.find('p').text.strip() 
                numgiri = numgiri + 1
    else:
        return directors
    return directors


In [588]:
def get_novel(doc2_page):
    
    description_tag = doc2_page.findAll('li', class_='profile')

    # Estrai il testo, se il tag è stato trovato
    if description_tag:
        for profile in description_tag:
            # Cerca un tag 'p' con classe 'character' che contiene la parola 'Director'
            if 'Novel' in profile.find('p', class_='character').text:
                # Se trovato, estrai il testo del tag 'p' precedente (il nome)
                return profile.find('p').text.strip() 
    else:
        return "Novel not found"


In [589]:
def get_writers(doc2_page):
    
    description_tag = doc2_page.findAll('li', class_='profile')
    # Estrai il testo, se il tag è stato trovato
    directors = "Writer not found"
    if description_tag:
        numgiri=0
        for profile in description_tag:
            # Cerca un tag 'p' con classe 'character' che contiene la parola 'Director'
            if 'Writer' in profile.find('p', class_='character').text:
                if numgiri == 0:
                    directors = profile.find('p').text.strip()
                else:
                    directors = directors + ',' + profile.find('p').text.strip() 
                numgiri = numgiri + 1
    else:
        return directors
    return directors


In [593]:
def get_screenplay(doc2_page):
    
    description_tag = doc2_page.findAll('li', class_='profile')
    # Estrai il testo, se il tag è stato trovato
    directors = "Screenplay not found"
    if description_tag:
        numgiri=0
        for profile in description_tag:
            # Cerca un tag 'p' con classe 'character' che contiene la parola 'Director'
            if 'Screenplay' in profile.find('p', class_='character').text:
                if numgiri == 0:
                    directors = profile.find('p').text.strip()
                else:
                    directors = directors + ',' + profile.find('p').text.strip() 
                numgiri = numgiri + 1
    else:
        return directors
    return directors


In [591]:
def get_state(doc2_page):
    # Cerca il tag `<strong>` che contiene il testo "Titolo originale"
    original_title_label = doc_2_list_.find('section', class_='facts left_column')
    p_tags_list = []

    # Controlla se la sezione è stata trovata
    if original_title_label:
        # Trova tutti i tag <p> all'interno della sezione
        p_tags = original_title_label.find_all('p')
        # Aggiungi il testo di ogni tag <p> alla lista
        for p_tag in p_tags:
            p_tags_list.append(p_tag.get_text().strip()) 
        return parsing_state(p_tags_list)
    else:
        return  "Stato non trovato"

def parsing_state(p_tags_list):
    # Lista per i valori estratti
    extracted_values = ['None','None','None','None','None']
    
    # Ciclo per estrarre i valori
    for item in p_tags_list:
        # Dividi la stringa una volta e prendi il secondo elemento
        parts = item.split(' ', 1)
        if len(parts) > 1:
            type = parts[0]
            value = parts[1]
            if type == 'Stato':
                extracted_values[1] = value
            if type=='Lingua':
                value = value.replace('Originale','')
                extracted_values[2] = value
            if type=='Titolo':
                value = value.replace('originale','')
                extracted_values[0] = value
            if type=='Budget':
                value = value.replace('$', '').replace(',', '').strip()
                extracted_values[3] = value
            if type=='Incasso':
                value = value.replace('$', '').replace(',', '').strip()
                extracted_values[4] = value
    return extracted_values

In [596]:
def get_keywords(doc2_page):
    keywords_section = doc2_page.find('section', class_='keywords right_column')
    keyword_lis = keywords_section.find_all('li')
    keywords = [li.get_text(strip=True) for li in keyword_lis]
    return keywords

In [580]:
def get_vote(doc_2_list_):
    tmpRating = doc_2_list_.find('div', class_='user_score_chart')['data-percent']
    return tmpRating

In [575]:
"""def get_title(doc_2_list_):
    # Cerca il tag <div> con classe 'title ott_true'
    tmp_title = doc_2_list_.find('div', class_='title ott_true')
    
    # All'interno di questo tag, cerca il tag <a> e ottieni il suo contenuto testuale
    title = tmp_title.find('a').text if tmp_title and tmp_title.find('a') else None
    return title"""

In [610]:
def get_title(doc_2_list_):
    # Questo è un esempio di come potrebbe apparire il tuo HTML. Dovresti utilizzare la tua variabile HTML effettiva qui.
    # Usa BeautifulSoup per effettuare il parsing dell'HTML
    soup = BeautifulSoup(doc_2_list_, 'html.parser')
    
    # Trova il tag <a> che contiene il titolo del film
    title_tag = soup.find('a', href="/movie/934632-rebel-moon-part-two-the-scargiver")
    
    # Estrai il testo del titolo
    movie_title = title_tag.get_text(strip=True) if title_tag else 'Titolo non trovato'
    
    print(movie_title)


In [604]:
"""def get_show_details(t_title, t_user_score, docs_2_list):
    # excuting a function here that empties the dictionary every time the function is called.
    scraped_data = []
    scraped_dict =  empty_dict()
    for i in range (0, len(t_title)):
        scraped_dict['Title'] = get_title(docs_2_list,i)
        scraped_dict['Genre'] = get_genres(docs_2_list, i)
        scraped_dict['Duration'] = get_duration(doc_2_list_,i)
        scraped_dict['Release_date'] = t_title[i].p.text   
        scraped_dict['User_rating'] = get_vote(doc_2_list_,i)
        tagline_info(docs_2_list, i, scraped_dict)        
        scraped_dict['Description'] = get_description(doc_2_list_,i)
        scraped_dict['Directors'] = get_directors(doc_2_list_,i)
        scraped_dict['Novel'] = get_novel(doc_2_list_,i)
        scraped_dict['Writers'] = get_writers(doc_2_list_,i)
        scraped_dict['Screenplay'] = get_writers(doc_2_list_,i)
        scraped_dict['State'] = get_state(doc_2_list_,i)[1]
        scraped_dict['Language'] = get_state(doc_2_list_,i)[2]
        scraped_dict['Budget'] = get_state(doc_2_list_,i)[3]
        scraped_dict['Revenue'] = get_state(doc_2_list_,i)[4]
        scraped_dict['Keywords'] = get_keywords(doc_2_list_,i)
        scraped_dict['Cast'] = get_show_cast(docs_2_list, i)
        scraped_data.append(scraped_dict)
    df = pd.DataFrame(scraped_data)
    return df
"""

link = 'https://www.themoviedb.org/movie/'
links = []
for i in range(10000,10010):
    links.append(link + str(i))
scraped_data = []
for i in range(0,len(links)):
    doc_2_list_ = get_page_content(links[i])
    scraped_dict =  empty_dict()
    scraped_dict['Title'] = get_title(doc_2_list_)
    scraped_dict['Genre'] = get_genres(doc_2_list_)
    scraped_dict['Duration'] = get_duration(doc_2_list_)
    # scraped_dict['Release_date'] = t_title[i].p.text   
    scraped_dict['User_rating'] = get_vote(doc_2_list_)
    # tagline_info(doc_2_list_, i, scraped_dict)        
    # scraped_dict['Description'] = get_description(docs_2_list,i)
    scraped_dict['Directors'] = get_directors(doc_2_list_)
    scraped_dict['Novel'] = get_novel(doc_2_list_)
    scraped_dict['Writers'] = get_writers(doc_2_list_)
    scraped_dict['Screenplay'] = get_screenplay(doc_2_list_)
    scraped_dict['State'] = get_state(doc_2_list_)[1]
    scraped_dict['Language'] = get_state(doc_2_list_)[2]
    scraped_dict['Budget'] = get_state(doc_2_list_)[3]
    scraped_dict['Revenue'] = get_state(doc_2_list_)[4]
    scraped_dict['Keywords'] = get_keywords(doc_2_list_)
    scraped_dict['Cast'] = get_show_cast(doc_2_list_)
    scraped_data.append(scraped_dict)
df = pd.DataFrame(scraped_data)
print(df)

                                          Title  \
0                                          None   
1                                          None   
2                                     Mona Lisa   
3                                      Il santo   
4                                          None   
5  Dietro le linee nemiche II - L'asse del male   
6                                          None   
7                                          None   
8                                          None   
9                           Koda, fratello orso   

                                               Genre  Duration Release_date  \
0                                 [Commedia, Dramma]       116         None   
1                           [Commedia, Fantascienza]        91         None   
2                 [Crime, Dramma, Romance, Thriller]       104         None   
3  [Thriller, Azione, Romance, Fantascienza, Avve...       112         None   
4       [Dramma, Fantasy, Horror, Thriller,

In [606]:
get_title(doc_2_list_)

'Koda, fratello orso'

In [605]:
df.to_csv('check.csv')
pd.read_csv('check.csv',index_col=[0])

,Title,Genre,Duration,Release_date,User_rating,Tagline,Description,Directors,Novel,Writers,Screenplay,State,Language,Budget,Revenue,Keywords,Cast
0,NaN,"['Commedia', 'Dramma']",116,NaN,73,NaN,NaN,Sergio Cabrera,NaN,"Humberto Dorado,Jorge Goldenberg,Frank Ramírez",Screenplay not found,Rilasciato,Spagnolo; Castigliano,-,-,"['roommate', 'pastor', 'squatter', 'anarchist'...","['Fausto Cabrera', 'Frank Ramírez', 'Delfina G..."
1,NaN,"['Commedia', 'Fantascienza']",91,NaN,51,NaN,NaN,Yahoo Serious,NaN,Writer not found,"Yahoo Serious,David Roach",Rilasciato,Inglese,24900000.00,5000000.00,"[""rock 'n' roll"", 'atomic bomb', 'nobel prize'...","['Yahoo Serious', 'Odile Le Clezio', 'Ian ""Pee..."
2,Mona Lisa,"['Crime', 'Dramma', 'Romance', 'Thriller']",104,NaN,69,NaN,NaN,Neil Jordan,NaN,Writer not found,"Neil Jordan,David Leland",Rilasciato,Inglese,-,5794184.00,"['london, england', 'prostitute', 'ex-detainee...","['Bob Hoskins', 'Cathy Tyson', 'Michael Caine'..."
3,Il santo,"['Thriller', 'Azione', 'Romance', 'Fantascienz...",112,NaN,61,NaN,NaN,Phillip Noyce,NaN,Writer not found,"Jonathan Hensleigh,Wesley Strick",Rilasciato,Inglese,68000000.00,118100000.00,"['london, england', 'berlin, germany', 'spy', ...","['Val Kilmer', 'Elisabeth Shue', 'Rade Šerbedž..."
4,NaN,"['Dramma', 'Fantasy', 'Horror', 'Thriller', 'M...",131,NaN,55,NaN,NaN,Mick Garris,Stephen King,Writer not found,Screenplay not found,Rilasciato,Inglese,12000000.00,-,"['sheriff', 'nevada', 'highway', 'psychopath',...","['Tom Skerritt', 'Steven Weber', 'Annabeth Gis..."
5,Dietro le linee nemiche II - L'asse del male,"['Azione', 'Avventura', 'Thriller']",96,NaN,47,NaN,NaN,James Dodson,NaN,Writer not found,Screenplay not found,Rilasciato,Inglese,-,-,"['nuclear missile', 'soldier']","['Matt Bushell', 'Denis Arndt', 'Ben Cross', '..."
6,NaN,"['Azione', 'Crime', 'Dramma']",98,NaN,39,NaN,NaN,James M. Hausler,NaN,James M. Hausler,Screenplay not found,Rilasciato,Inglese,-,-,"['prison', 'ex-detainee', 'hoodlum']","['Richard Roundtree', 'Robert Loggia', 'Lucie ..."
7,NaN,"['Horror', 'Thriller']",84,NaN,54,NaN,NaN,Gregory Dark,NaN,Dan Madigan,Screenplay not found,Rilasciato,Inglese,8000000.00,18700000.00,"['hotel', 'prisoner', 'eye ball', 'remake', 'm...","['Glenn Jacobs', 'Christina Vidal', 'Luke Pegl..."
8,NaN,"['Horror', 'Thriller', 'Mistero']",83,NaN,56,NaN,NaN,Courtney Solomon,Brent Monahan,Courtney Solomon,Screenplay not found,Rilasciato,Inglese,14000000.00,29612137.00,"['witch', 'curse', 'suitor', 'family']","['Donald Sutherland', 'Rachel Hurd-Wood', 'Sis..."
9,"Koda, fratello orso","['Avventura', 'Animazione', 'Famiglia']",85,NaN,73,NaN,NaN,"Robert Walker,Aaron Blaise",NaN,Writer not found,"Steve Bencich,David Hoselton,Lorne Cameron,Ron...",Rilasciato,Inglese,128000000.00,250397798.00,"['sibling relationship', 'grizzly bear', 'inui...","['Joaquin Phoenix', 'Jeremy Suarez', 'Jason Ra..."


In [563]:
def get_show_details(t_title, t_user_score, docs_2_list):
    # excuting a function here that empties the dictionary every time the function is called.
    scraped_data = []
    scraped_dict =  empty_dict()
    for i in range (0, len(t_title)):
        scraped_dict['Title'] = get_title(docs_2_list,i)
        scraped_dict['Genre'] = get_genres(docs_2_list, i)
        scraped_dict['Duration'] = get_duration(docs_2_list,i)
        scraped_dict['Release_date'] = t_title[i].p.text   
        scraped_dict['User_rating'] = get_vote(docs_2_list,i)
        tagline_info(docs_2_list, i, scraped_dict)        
        scraped_dict['Description'] = get_description(docs_2_list,i)
        scraped_dict['Directors'] = get_directors(docs_2_list,i)
        scraped_dict['Novel'] = get_novel(docs_2_list,i)
        scraped_dict['Writers'] = get_writers(docs_2_list,i)
        scraped_dict['Screenplay'] = get_writers(docs_2_list,i)
        scraped_dict['State'] = get_state(docs_2_list,i)[1]
        scraped_dict['Language'] = get_state(docs_2_list,i)[2]
        scraped_dict['Budget'] = get_state(docs_2_list,i)[3]
        scraped_dict['Revenue'] = get_state(docs_2_list,i)[4]
        scraped_dict['Keywords'] = get_keywords(docs_2_list,i)
        scraped_dict['Cast'] = get_show_cast(docs_2_list, i)
        scraped_data.append(scraped_dict)
    df = pd.DataFrame(scraped_data)
    return df


In [568]:
tag_title_, tag_user_score_, doc_2_list_ = get_show_info(doc)

In [ ]:
x = get_show_details(tag_title_, tag_user_score_, doc_2_list_)
x.to_csv('check.csv')
# pd.read_csv('check.csv',index_col=[0])

In [ ]:
# Parsing dell'HTML con BeautifulSoup
# soup = BeautifulSoup(doc_2_list_[0], 'lxml')
# Trovare tutti gli elementi <li> con classe "profile"
print(doc_2_list_[0])
profiles = doc_2_list_[0].find_all('li', class_='profile')

# Lista per mantenere gli attori e i loro ruoli
actors_and_characters = []

# Iterare su ciascun profilo per estrarre nome e ruolo
for profile in profiles:
    name_tag = profile.find('a')  # Trova il tag <a> per il nome
    character_tag = profile.find('p', class_='character')  # Trova il tag <p> per il personaggio

    # Controlla che entrambi i tag siano stati trovati prima di procedere
    if name_tag is not None and character_tag is not None:
        actor_name = name_tag.text.strip()  # Testo del nome dell'attore
        character_name = character_tag.text.strip()  # Testo del personaggio
        actors_and_characters.append((actor_name, character_name))
    else:
        # Opzionale: stampa un messaggio o gestisci il caso in cui l'informazione non sia completa
        print("Missing actor name or character information.")

# Stampa i risultati
#for actor, character in actors_and_characters:
    #print(f"Name: {actor}, Job: {character}")


NameError: name 't_title' is not defined